In [2]:
library(tidyverse)
library(ISLR)
library(nnet)

Warning message:
"package 'ISLR' was built under R version 3.4.2"

In [4]:
#Look over the data
data(iris)
head(iris)
str(iris)

Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
5.1,3.5,1.4,0.2,setosa
4.9,3.0,1.4,0.2,setosa
4.7,3.2,1.3,0.2,setosa
4.6,3.1,1.5,0.2,setosa
5.0,3.6,1.4,0.2,setosa
5.4,3.9,1.7,0.4,setosa


'data.frame':	150 obs. of  5 variables:
 $ Sepal.Length: num  5.1 4.9 4.7 4.6 5 5.4 4.6 5 4.4 4.9 ...
 $ Sepal.Width : num  3.5 3 3.2 3.1 3.6 3.9 3.4 3.4 2.9 3.1 ...
 $ Petal.Length: num  1.4 1.4 1.3 1.5 1.4 1.7 1.4 1.5 1.4 1.5 ...
 $ Petal.Width : num  0.2 0.2 0.2 0.2 0.2 0.4 0.3 0.2 0.2 0.1 ...
 $ Species     : Factor w/ 3 levels "setosa","versicolor",..: 1 1 1 1 1 1 1 1 1 1 ...


In [39]:
# Split into training and testing splits
set.seed(123)
ind <- sample(2, nrow(iris), replace = TRUE, prob = c(.3, .7))
train <- iris[ind == 2,]
test <- iris[ind == 1,]
testX <- test[,-5]
testY <- test[,5]


[1] 44

[1] 106

# Classification Algorithms

### Multinomial Logistic Regression

What is it? 

When the response has more than two categories we can't use logistic regression anymore and we need to use multinomial logistic regression.

This is a GLM where the random component assumes that the distribution of Y is  Multinomial(n,π), where π is a vector with probabilities of "success" for each category.

If the response is order for exampe Low, Medium, and High then we will use Ordinal Logistic Regression but we will look at the unordered case.

How we assess if the model is good or not is by using the goodness of fit test. 

Goodness of fit test compares the observed values to the expected (fitted or predicted) values. Most often the observed data represent the fit of the saturated model, the most complex model possible with the given data. Thus, most often the alternative hypothesis (HA) will represent the saturated model MA which fits perfectly because each observation has a separate parameter. 

In practice, it's a good idea to compute both X2 and G2 to see if they lead to similar results. If the resulting p-values are close, then we can be fairly confident that the large-sample approximation is working well.

In [27]:
fit1 <- multinom(Species~., train)
summary(fit1)

# weights:  18 (10 variable)
initial  value 48.338941 
iter  10 value 6.740553
iter  20 value 4.683654
iter  30 value 4.622101
iter  40 value 4.613563
iter  50 value 4.610597
iter  60 value 4.610556
iter  70 value 4.610542
iter  80 value 4.610458
iter  90 value 4.610339
iter 100 value 4.610243
final  value 4.610243 
stopped after 100 iterations


Call:
multinom(formula = Species ~ ., data = train)

Coefficients:
           (Intercept) Sepal.Length Sepal.Width Petal.Length Petal.Width
versicolor    13.40180    -2.855247   -8.170263     9.737625   0.1967187
virginica    -14.87891    -5.201302  -10.814825    16.341011  10.7361211

Std. Errors:
           (Intercept) Sepal.Length Sepal.Width Petal.Length Petal.Width
versicolor    11.84296     13.44375    24.85049     45.43206    27.70191
virginica     12.72187     13.45321    25.15859     45.62134    28.19950

Residual Deviance: 9.220486 
AIC: 29.22049 

In [28]:
z <- summary(fit1)$coefficients/summary(fit1)$standard.errors
z

,(Intercept),Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
versicolor,1.131625,-0.2123848,-0.3287768,0.2143338,0.007101269
virginica,-1.169554,-0.3866216,-0.4298661,0.3581879,0.380720267


In [29]:
p <- (1 - pnorm(abs(z), 0, 1)) * 2
p

,(Intercept),Sepal.Length,Sepal.Width,Petal.Length,Petal.Width
versicolor,0.2577920,0.8318068,0.7423244,0.8302868,0.9943341
virginica,0.2421805,0.6990363,0.6672931,0.7202027,0.7034108


#### What this means

We have three levels for response variabel which is setosa versicolor and virginica. 

In [47]:
mult.pred <- predict(fit1, testX)
table(mult.pred, testY)

            testY
mult.pred    setosa versicolor virginica
  setosa         15          0         0
  versicolor      0         12         1
  virginica       0          2        14

### LDA and QDA

#### What is it?

A supervised machine learning method for classification when you have more than two categories for the response. We use this over the logistic regression method in case the categories are very well seperated and the data is fairly small. With bigger datasets we will see how it stacks up with RandomForest but in general QDA works better than LDA.

#### How it makes probability predictions for each of the categories?

This method uses Bayes theorem to calculate the probability for each of the categories given the predictor variables for each of the observations. The classiﬁer plugs the estimates  and assigns an observation X = x to the class for which the discriminant function is largest. QDA has a quadratic one and LDA has a linear discriminant function.Lastly LDA and QDA assumes that the observations within each class are drawn from a multivariate Gaussian distribution with a class speciﬁc mean vector but only LDA assumes a covariance matrix that is common to all K classes. QDA relaxes this assumption and does not assume this so covariance matrix are different with all k classes.


In [40]:
library (MASS)
lda.fit <- lda(Species~., train)
lda.fit

Call:
lda(Species ~ ., data = train)

Prior probabilities of groups:
    setosa versicolor  virginica 
 0.3301887  0.3396226  0.3301887 

Group means:
           Sepal.Length Sepal.Width Petal.Length Petal.Width
setosa         4.940000    3.405714     1.445714   0.2428571
versicolor     5.908333    2.766667     4.202778   1.3027778
virginica      6.634286    2.925714     5.565714   2.0428571

Coefficients of linear discriminants:
                    LD1        LD2
Sepal.Length  0.5329061  0.2942187
Sepal.Width   2.1256776  1.9325511
Petal.Length -1.9624296 -1.1434715
Petal.Width  -3.5611214  3.0035717

Proportion of trace:
  LD1   LD2 
0.993 0.007 

In [42]:
lda.pred <- predict(lda.fit, testX)
table(lda.pred$class, testY)

            testY
             setosa versicolor virginica
  setosa         15          0         0
  versicolor      0         13         1
  virginica       0          1        14

In [43]:
qda.fit <- qda(Species~., train)
qda.fit

Call:
qda(Species ~ ., data = train)

Prior probabilities of groups:
    setosa versicolor  virginica 
 0.3301887  0.3396226  0.3301887 

Group means:
           Sepal.Length Sepal.Width Petal.Length Petal.Width
setosa         4.940000    3.405714     1.445714   0.2428571
versicolor     5.908333    2.766667     4.202778   1.3027778
virginica      6.634286    2.925714     5.565714   2.0428571

In [44]:
qda.pred <- predict(qda.fit, testX)
table(qda.pred$class, testY)

            testY
             setosa versicolor virginica
  setosa         15          0         0
  versicolor      0         10         1
  virginica       0          4        14

### Classification Trees and RandomForest

### XgBoost

### Naive Bayes

### Support Vector Machines

# Regression Algorithms

### Multiple Linear Regression

### Ridge Lasso and Elastic Net

### Regrssion RandomForest

### Least-Angle Regression (LARS)

### Stepwise Regression

### Multivariate Adaptive Regression Splines 

### Locally Estimated Scatterplot Smoothing